<b> Segmenting and Clustering Neighborhoods in Toronto <b>


In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


<b> Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe </b>

In [9]:
# Import
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


<b> The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood </b>

In [10]:
# Source Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Parse to html
html_parsed_result = BeautifulSoup(requests.get(url).text, 'html.parser')

# Parse wikipedia table
nb_tb = html_parsed_result.find('table', class_ = 'wikitable')

# Find rows
nb_rw = nb_tb.find_all('tr')

# extract 'Postcode', 'Borough', 'Neighbourhood' from wikipedia table
nb_info = []

# Append data to nb_info list
for row in nb_rw:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    nb_info.append(info)    
nb_info[0][-1] = 'Neighborhood'

# Create dataframe from nb_info list
nb_df = pd.DataFrame(nb_info[1:], columns=nb_info[0])

<b> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park </b>

In [14]:
# Clean by replacing not assigned values in Neighborhood column to the same as Borough in the cell
nb_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
for i in nb_not_assigned:
    nb_df['Neighborhood'][i] = nb_df['Borough'][i]

<b> Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. </b>

In [15]:
# Clean by droping unassigned cells
boroughs_not_assigned = nb_df.index[nb_df['Borough'] == 'Not assigned']
neighborhoods_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
neighborhoods_borough_not_assigned = boroughs_not_assigned & neighborhoods_not_assigned
nb_df.drop(nb_df.index[boroughs_not_assigned], inplace=True)
nb_df.reset_index(drop=True, inplace=True)

# Print top 10
nb_df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<b> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</b>

In [16]:
# Group same borough
gp = nb_df.groupby('Postcode')
gp_nb = gp['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
gp_boroughs = gp['Borough'].apply(lambda x: set(x).pop())
gp_df = pd.DataFrame(list(zip(gp_boroughs.index, gp_boroughs, gp_nb)))
gp_df.columns = ['Postcode', 'Borough', 'Neighborhood']


# Print top 10
gp_df.head(12)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<b>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. </b>

In [24]:
print('The DataFrame shape is', gp_df.shape)

The DataFrame shape is (103, 3)


In [25]:
gp_df.to_csv('Segmenting_Clustering_Neighborhoods_Toronto_sec1.csv',index=False)

<b> given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data </b>

In [29]:
data_csv = "Segmenting_Clustering_Neighborhoods_Toronto_sec1.csv"
coordinate_df = pd.read_csv(data_csv)
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv)
toronto_df = coordinate_df.join(geocsv_data)
toronto_df.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [30]:
toronto_df.to_csv('Segmenting_Clustering_Neighborhoods_Toronto_sec2.csv',index=False)
toronto_df.shape

(103, 6)